## Metrics and additional features

In [16]:
import os
import sys
from collections import defaultdict

from tabulate import tabulate

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    
from source.core import artifacts, settings
from source.utils.params import params_from_path
from source.utils.utils import write_txt


In [17]:
# Display table with only one metric per machine learning algo

metrics_all = artifacts.load_metrics()

# Dictionary to store the values
data = defaultdict(dict)
metric_wanted = "balanced_accuracy_score"

for key, metrics_ in metrics_all.items():
    value = getattr(next((m for m in metrics_ if m.name == metric_wanted), None), 'mean', 0)
    if value == 0:
        continue

    params = params_from_path(key)
    if params.load_imagings_ids != [1, 2, 3]:
        continue
    if params.load_cameras_labels != ["vnir", "swir"]:
        continue
    if not params.estimator_is_optimized:
        continue

    estimator = params.estimator_name
    dataset_id = params.load_group_id
    data[dataset_id][estimator] = value

# Convert dictionary to table format
headers = ["Dataset (D#)"] + sorted({estimator for estimators in data.values() for estimator in estimators})
rows = [[dataset_id] + [data[dataset_id].get(estimator, "") for estimator in headers[1:]] for dataset_id in sorted(data)]

table = tabulate(rows, headers, tablefmt="grid")
write_txt(table, settings.outputs_dir / f"table__{metric_wanted}.txt")